#Projeto COVID-19
##Digital Innovation One

In [ ]:
#Importando bibliotecas
import pandas as pd
import numpy as np
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
#Importar dados do GitHub
url = 'https://github.com/neylsoncrepalde/projeto_eda_covid/blob/master/covid_19_data.csv?raw=true'

In [ ]:
#Criando DataFrame
df = pd.read_csv(url, parse_dates=['ObservationDate', 'Last Update'])

In [ ]:
#Verificando tipos de dados
df.dtypes

SNo                         int64
ObservationDate    datetime64[ns]
Province/State             object
Country/Region             object
Last Update        datetime64[ns]
Confirmed                 float64
Deaths                    float64
Recovered                 float64
dtype: object

In [ ]:
#Função para corrigir nomes com regex
import re

def corrige_colunas(col_name):
    return re.sub(r"[/| ]", "", col_name).lower()

In [ ]:
df.columns = [corrige_colunas(col) for col in df.columns]

In [ ]:
#Verificar casos por país
df.countryregion.value_counts()

In [ ]:
#Verificar todos os paises
df.countryregion.unique()

In [ ]:
#Filtrar dados do Brasil
df.loc[df.countryregion == 'Brazil']

In [ ]:
brasil = df.loc[
    (df.countryregion == 'Brazil') &
    (df.confirmed > 0)]
brasil

##Casos confirmados no Brasil

In [15]:
# Gráfico da evolução de casos confirmados no brasil
px.line(brasil, 'observationdate', 'confirmed', title='Casos confirmados no Brasil')

##Aumento de casos por dia

In [ ]:
#Programação funcional, função lambda
brasil['novoscasos'] = list(map(
    lambda x: 0 if (x==0) else brasil['confirmed'].iloc[x] - brasil['confirmed'].iloc[x-1],
    np.arange(brasil.shape[0])
))


In [17]:
px.line(brasil, 'observationdate', 'novoscasos', title='Novos casos por dia confirmados no Brasil')

##Mortes no Brasil

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=brasil.observationdate, y=brasil.deaths, name='Mortes',
               mode='lines+markers', line={'color':'red'})
)

#Layout
fig.update_layout(title='Mortes por COVID-19 no Brasil')
fig.show()

##Taxa de crescimento de mortes
####A taxa_crescimento = (presente/passado)**(1/n)-1

In [23]:
#Se a data inicio for None, define como a primeira data disponível
def taxa_crescimento(base, variavel, data_inicio=None, data_fim=None):
  if data_inicio == None:
    data_inicio = base.observationdate.loc[base[variavel] > 0].min()
  else:
    data_inicio = pd.to_datetime(data_inicio)

  if data_fim == None:
    data_fim = base.observationdate.iloc[-1]
  else:
    data_fim = pd.to_datetime(data_fim)

  #Define os valore do presente e passado
  passado = base.loc[base.observationdate == data_inicio, variavel].values[0]
  presente = base.loc[base.observationdate == data_fim, variavel].values[0]

  #Define o número de pontos no tempo que vamos avaliar
  n = (data_fim - data_inicio).days

  #Caucular a taxa
  taxa = (presente / passado)**(1/n)-1

  return taxa*100

In [24]:
#Taxa de crescimento médio do COVID no Brasil em todo o período
taxa_crescimento(brasil, 'confirmed')

16.27183353112116

##Taxa de crescimento diário do COVID-19 no Brasil

In [25]:
def taxa_crescimento_diaria(base, variavel, data_inicio=None):
  if data_inicio == None:
    data_inicio = base.observationdate.loc[base[variavel] > 0].min()
  else:
    data_inicio = pd.to_datetime(data_inicio)

  data_fim = base.observationdate.max()

  #Define o número de pontos no tempo que vamos avaliar
  n = (data_fim - data_inicio).days

  #Taxa calculada de um dia para o outro
  taxas = list(map(
      lambda x: (base[variavel].iloc[x] - base[variavel].iloc[x-1]) / base[variavel].iloc[x-1],
      range(1, n+1)
  ))
  return np.array(taxas) * 100

In [ ]:
tx_dia = taxa_crescimento_diaria(brasil, 'confirmed')
tx_dia

In [ ]:
primeiro_dia = brasil.observationdate.loc[brasil.confirmed > 0].min()

px.line(x=pd.date_range(primeiro_dia, brasil.observationdate.max())[1:],
        y=tx_dia, title='Taxa de crescimento de casos confirmados no Brasil')

##Predições

In [37]:
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.pyplot as plt
from IPython.core.pylabtools import figsize

In [ ]:
#Novo DataFrame de casos confirmados por dia
casos_confirmados = brasil.confirmed
casos_confirmados.index = brasil.observationdate
casos_confirmados

In [39]:
resultado = seasonal_decompose(casos_confirmados)

In [ ]:
fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1)

ax1.plot(resultado.observed)
ax2.plot(resultado.trend)
ax3.plot(resultado.seasonal)
ax4.plot(casos_confirmados.index, resultado.resid)
ax4.axhline(0, linestyle='dashed', c='black')
plt.show

##Modelo ARIMA (Média móvel integrada autoregressiva)
##Tenta modelar o futuro em função do passado

In [ ]:
#instalando o pacote ARIMA
! pip install pmdarima

In [48]:
from pmdarima.arima import auto_arima
modelo = auto_arima(casos_confirmados)

In [ ]:
fig = go.Figure(go.Scatter(
    x=casos_confirmados.index, y=casos_confirmados, name='Observados'
))

fig.add_trace(go.Scatter(
    x=casos_confirmados.index, y=modelo.predict_in_sample(), name='Predição'
))
fig.add_trace(go.Scatter(
    x=pd.date_range('2020-05-20', '2020-06-20'), y=modelo.predict(31), name='Forecast'
))
fig.update_layout(title='Previsão de casos confirmados no Brasil para os próximos 30 dias')
fig.show()